In [1]:
import tensorflow as tf
import numpy as np
import h5py as h5
import os
import re
import pandas as pd

In [2]:
def tf_cov(H):
    (n,p)=H.shape
    X=tf.placeholder(dtype=tf.float32,shape=(n,p))
    cd=tf.constant([1/(n-1.0)],dtype=tf.float32)
    tfM=tf.expand_dims(tf.reduce_mean(X,0),0)
    cX=X-tfM
    =tf.matmul(cX,cX,transpose_a=True,transpose_b=False)
    Z=Y*cd
    with tf.Session() as session:
        return(session.run(Z,feed_dict={X:H}))

In [4]:
tx=np.array([[4,3,2],[7,9,6],[8,1,5]],dtype=np.float32)
tx

array([[ 4.,  3.,  2.],
       [ 7.,  9.,  6.],
       [ 8.,  1.,  5.]], dtype=float32)

In [57]:
tf_cov(H)[0:4,0:4]

array([[ 0.11189754,  0.0118662 , -0.1022286 ,  0.00867928],
       [ 0.0118662 ,  0.08398195, -0.0109462 , -0.00109689],
       [-0.1022286 , -0.0109462 ,  0.10440128, -0.00983445],
       [ 0.00867928, -0.00109689, -0.00983445,  0.1502285 ]], dtype=float32)

In [7]:
print (tf.__version__)

1.4.0


In [20]:
m=tf.placeholder(tf.float32,shape=0)
Ne=tf.placeholder(tf.float32,shape=0)
NNe=tf.multiply(tf.constant([4],shape=(0,),dtype=tf.float32),Ne)
scalar = tf.convert_to_tensor(NNe, dtype=tf.float32, name="scalar")
shape = scalar.get_shape()
shape

TensorShape([Dimension(0)])

In [ ]:
import edward as ed
import numpy as np
import tensorflow as tf

from edward.models import Normal

In [8]:
h5file = '/home/nwknoblauch/Desktop/scratch/polyg_scratch/h5/scz2_seq_hapmap_haplo.h5'
hf = h5.File(h5file,mode='r')
hf

<HDF5 file "scz2_seq_hapmap_haplo.h5" (mode r)>

In [63]:
n_p=5000
H=hf['dosage'][:,0:n_p]
print(H.shape)
(n,p)=H.shape
mapd=hf['/SNPinfo/map'][0:p]
m=1/(2*85)
Ne=4*11490.672741/100
theta=0.001029112
cd=1/(n-1.0)

(1006, 5000)


In [74]:
%%time
#m=tf.placeholder(tf.float32,shape=0)
#Ne=tf.placeholder(tf.float32,shape=0)

X=tf.placeholder(dtype=tf.float32,shape=(n,p))
tfM=tf.expand_dims(tf.reduce_mean(X,0),0)
cX=X-tfM
Y=tf.matmul(cX,cX,transpose_a=True,transpose_b=False)
Z=tf.scalar_mul(cd,Y)
mapdata=tf.placeholder(tf.float32,shape=(p))
mapdatb=tf.placeholder(tf.float32,shape=(p))
distmat=tf.abs(tf.expand_dims(mapdata,0)-tf.expand_dims(mapdatb,1))
rho=tf.scalar_mul(Ne,distmat)
rho=tf.scalar_mul(m,-rho)
shrinkage=tf.exp(rho)
nshrinkage=shrinkage-tf.diag(tf.diag_part(shrinkage))+tf.eye(p)
S=shrinkage*Z
eyem=tf.scalar_mul((0.5*theta*(1.0-0.5*theta)),tf.eye(p))
SigHat=tf.scalar_mul((1-theta)*(1-theta),S)+eyem
mvars=1/tf.sqrt(tf.diag_part(SigHat))
ret=SigHat*tf.expand_dims(mvars,1)*tf.expand_dims(mvars,0)
Q,D=tf.self_adjoint_eig(ret)
with tf.Session() as session:
   [rQ,rD]=session.run([Q,D],feed_dict={X:H,
                                 mapdata:mapd,mapdatb:mapd})
print(rD.shape)

(5000, 5000)
CPU times: user 14.9 s, sys: 40.7 s, total: 55.6 s
Wall time: 55.1 s


In [69]:
rret[0:4,0:4]

array([[ 1.        ,  0.119789  , -0.91377622,  0.03324893],
       [ 0.119789  ,  1.00000012, -0.11470513, -0.00492616],
       [-0.91377616, -0.11470512,  1.        , -0.04017282],
       [ 0.03324893, -0.00492616, -0.04017282,  1.00000012]], dtype=float32)

array([[ 0.11218164,  0.0116507 , -0.09903204,  0.00431928],
       [ 0.0116507 ,  0.08432347, -0.01077785, -0.00055482],
       [-0.09903204, -0.01077785,  0.1047008 , -0.00504173],
       [ 0.00431928, -0.00055482, -0.00504173,  0.15043375]], dtype=float32)

array([[ 0.11189754,  0.0118662 , -0.1022286 ,  0.00867928],
       [ 0.0118662 ,  0.08398195, -0.0109462 , -0.00109689],
       [-0.1022286 , -0.0109462 ,  0.10440128, -0.00983445],
       [ 0.00867928, -0.00109689, -0.00983445,  0.1502285 ]], dtype=float32)

0.0009950248756218905